<div style="border-bottom: 2px solid #5c5c5c33;">
    <center style="font-size: 30px;margin-bottom: 20px;">بسم الله الرحمن الرحیم</center>
    <br>
    <center style="font-size:20px; padding-bottom: 20px;line-height: 2;"> طراحی سامانه های یادگیری ماشین <br> تمرین اول</center>
    <div style="font-size:17px; direction:rtl;padding-bottom: 15px;">نام و نام خانوادگی : زینب احیائی</div>
    <div style="font-size:17px; direction:rtl; padding-bottom: 15px;">شماره دانشجویی : 400201061</div>
</div>

<div style="direction:rtl">
    <h2 style="padding-bottom: 20px;">آماده‌سازی داده</h2>
    <div style="font-size: 16px;line-height: 1.6;">
        در این بخش ابتدا داده‌های خام مربوط به ۱۲۲۱ محصول از طریق خزش در وب سایت <a href="https://divar.ir/">دیوار</a>
     در دسته کالاهای دیجیتال و در زیر دسته‌های موبایل و تبلت، رایانه، کنسول و بازی ویدئویی و آنلاین  
        و صوتی و تصویری  و از بین آگهی‌های شهر‌های تهران، مشهد، شیراز و اصفهان جمع آوری شده است.
        این کار با استفاده از <a href="https://scrapy.org/">scrapy</a>
        و در دو مرحله انجام شده است.
        در مرحله اول آدرس‌های این محصولات به دست آمده و در مرحله بعد اطلاعات محصول از داخل این صفحات استخراج شده است.
        برای استخراج آدرس‌ها از spider ای با ساختار زیر استفاده شده است:
    </div>
</div>

In [ ]:
import scrapy
from scrapy.selector import Selector
from selenium import webdriver
import time

class URLSpider(scrapy.Spider):
    name = "urlspider"
    allowed_domains = ["divar.ir"]
    start_urls = ["https://divar.ir/s/mashhad/mobile-tablet"]
    
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.count = 0


    def closed(self, reason):
        self.driver.quit()


    def parse(self, response):
        self.driver.get(response.url)

        # Scroll down and load more data
        while True:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            sel = Selector(text=self.driver.execute_script("return document.body.innerHTML;"))

            for post in sel.css('div.post-card-item-af972'):
                self.count = self.count + 1
                href = post.css('a:nth-child(1)::attr(href)').extract()

                yield {
                    'href': href,
                }
            
            if self.count > 200 :
                break

        self.driver.quit()

<div style="font-size: 16px;line-height: 1.6;direction:rtl">
      برای استخراج اطلاعات محصول نیز از spider ای با ساختار زیر استفاده شده است:
</div>


In [21]:
import json
import time
from scrapy import Spider, Request, Selector

class DataSpider(Spider):
    name = 'dataspider'
   
    def start_requests(self):
        with open('/Users/zeinab/Desktop/MLSD_HW1/scrapy/divar/urls/mashhad-mobile-tablet.json', 'r') as f:
            urls = json.load(f)
        for url in urls:
            time.sleep(1)
            yield Request(url="https://divar.ir" + url["href"][0], callback=self.parse)

    def parse(self, response):
        price = ''
        for value in response.css('p.kt-unexpandable-row__value'):
            if('تومان' in value.css("::text").get().strip()):
                price = value.css("::text").get().strip()

        img_div = response.css('div.kt-carousel__thumbnails')
        img_count = len(img_div.css('img'))

        yield {
            'title' : response.css('div.kt-page-title__title::text').get(),
            'desc' : response.css('p.kt-description-row__text::text').get(),
            'brand' : response.css('a.kt-unexpandable-row__action::text').get(),
            'image-count' : max(1, img_count),
            'price' : price,
            'city' : 'Mashhad',
            'category' : 'mobile-tablet'
        }

<div style="font-size: 16px;line-height: 1.6;direction:rtl">
      در نهایت نتایج در یک فایل json با فرمتی مشابه زیر ذخیره شده است:
</div>


In [ ]:
[
    {
    "title": "سینما خانگی سامسونگ ۱۰۰۰ وات، ساخت اندونزی",
    "desc": "سینما خانگی ۷ تیکه دارای ۴ باند یک ساب، یک DVD player ، یک باند کوچک رو میزی، حدود ۲ ساله استفاده نشده، بسیار تمیز",
    "brand": null,
    "image-count": 4,
    "price": "۳٬۷۰۰٬۰۰۰ تومان",
    "city": "Tehran",
    "category": "video-audio-device"
  }
]


<div style="direction:rtl">
    <h3 style="padding-bottom: 20px; direction: rtl;">تبدیل داده به ساختار جدولی</h3>
    <div style="font-size: 16px;line-height: 1.6;">
   در این بخش داده‌ها به فرمت جدولی تبدیل شده و ساختار جدول آن در sql شرح داده می‌شود.
    </div>
</div>

In [ ]:
import json
import csv

# Open the JSON file
with open('data.json', 'r') as json_file:
    data = json.load(json_file)

# Open the CSV file and write headers
with open('data.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['id', 'name', 'age'])

    # Write data rows to CSV file
    for item in data:
        writer.writerow([item['id'], item['name'], item['age']])

In [ ]:
nltk.download('vader_lexicon')
!pip install rake-nltk
import json
import nltk
import string
import numpy as np
import pandas as pd 
import itertools
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from rake_nltk import Rake
import collections
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import metrics
from sklearn.pipeline import Pipeline

<div style="direction:rtl">
    <h2 style="padding-bottom: 20px;">تحلیل اکتشافی داده</h2>
    <div style="font-size: 16px;line-height: 1.6;">
      در این بخش به بررسی بیشتر داده می‌پردازیم.
    </div>
    <h3 style="padding-bottom: 20px;">تحلیل اکتشافی داده</h3>
</div>

<div style="direction:rtl">
    <h3 style="padding-bottom: 20px;">پاک‌سازی داده</h3>
    <div style="font-size: 16px;line-height: 1.6;">
      
    </div>
</div>

In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("divar_posts_dataset.csv")
df

,Unnamed: 0,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year
0,0,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,کلاسیک و شیک و استثنایی\nچرم مالزی\nچوب راش\nف...,54761638662241,5,NaN,mobile,3850000,ست مبلمان و نهارخوری ٩ نفره,NaN,NaN
1,1,False,NaN,for-the-home,furniture-and-home-decore,antiques-and-art,Mashhad,Tuesday 07PM,"سلام,یک عدد گلدون نخل سه طبقه ی سالم دارم با پ...",42727631379133,1,NaN,mobile,30000,گلدون مصنوعی نخل,NaN,NaN
2,2,False,NaN,vehicles,cars,heavy,Mashhad,Tuesday 07PM,سریال 43j$NUMبدون شکستگی سه حلقه لاستیک نو یک ...,63194439667302,4,NaN,mobile,-1,لودر کاتر پیلار 950,NaN,NaN
3,3,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Tehran,Tuesday 07PM,مبل راحتی هفت نفره شامل سه نفره یک عدد دونفره ...,19133025491169,4,NaN,mobile,600000,مبل راحتی هفت نفره بامیز جلو مبلی,NaN,NaN
4,4,False,NaN,personal,baby-and-toys,personal-toys,Karaj,Tuesday 08PM,شارژی کنترلی سویچ حمل تا 35 کیلو صندلی برای دو...,58998998335444,2,NaN,mobile,450000,ماشین شارژی,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947630,947630,True,NaN,for-the-home,building-and-garden,stove-and-heating,Shiraz,Friday 07AM,دوعدد یکی یک متری و دیگری 1.4 متری موجود است\n...,16127180351093,1,NaN,web,180000,رادیاتور پنلی نو ترک,NaN,NaN
947631,947631,True,NaN,electronic-devices,audio-video,camera-camcoders,Tehran,Friday 02PM,دوربین هندی کم کنون مدل لگریا fs36\n\nعکاسی و ...,29805298491725,1,NaN,mobile,350000,دوربین هندی کم کنون,NaN,NaN
947632,947632,True,پراید صندوق‌دار::Pride,vehicles,cars,light,Shiraz,Friday 04PM,پراید دوگانه کارخانه میباشد و کم کارکرد.روکش ....,55514317985619,3,43000.0,mobile,17400000,پراید دوگانه کارخانه مدل93,NaN,1393
947633,947633,True,NaN,electronic-devices,mobile-tablet,mobile-tablet-accessories,Mashhad,Friday 05PM,مونو پد یانگ تد همراه با پایه و اینه عقبش\nواس...,65938921533442,1,NaN,mobile,40000,مونوپد یانگ تد خیلی تمیز و نو,NaN,NaN


In [9]:
df['city'].unique()

array(['Tehran', 'Mashhad', 'Karaj', 'Qom', 'Isfahan', 'Shiraz', 'Tabriz',
       'Ahvaz', 'Kermanshah'], dtype=object)